In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.neural_network import  MLPClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,...,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent
0,ff8081815733a243015733b2876600a6,BHK2,"Cauvery Colony,Koramangala",21-04-2018 14:44,12.934471,77.634471,FAMILY,1,1,1,...,2,E,2,6,12,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",CORP_BORE,AP,2,40000
1,ff8081815ee25e15015ee50004da2acd,BHK2,Bellandur,09-10-2017 12:48,12.929557,77.672280,ANYONE,0,1,0,...,2,NE,2,3,4,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORPORATION,AP,2,22000
2,ff80818163f29c9e0163f46770b873e7,BHK3,Thiruvanmiyur,12-06-2018 22:02,12.982870,80.262012,FAMILY,0,1,0,...,3,E,3,1,5,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORP_BORE,AP,3,28000
3,ff80818164b68a700164b74b1b247a1d,BHK1,Attiguppe,20-07-2018 16:22,12.955991,77.531634,FAMILY,0,0,0,...,1,E,1,1,2,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,0,8000
4,ff80818163b1faf00163b4b9b8a163b1,BHK3,Kodihalli,31-05-2018 17:41,12.963903,77.649446,FAMILY,0,0,0,...,3,E,4,0,0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,1,45000


In [3]:
data.isnull().sum()

id                 0
type               0
locality           0
activation_date    0
latitude           0
longitude          0
lease_type         0
gym                0
lift               0
swimming_pool      0
negotiable         0
furnishing         0
parking            0
property_size      0
property_age       0
bathroom           0
facing             0
cup_board          0
floor              0
total_floor        0
amenities          0
water_supply       0
building_type      0
balconies          0
rent               0
dtype: int64

In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
data.shape

(20500, 25)

In [6]:
data.columns

Index(['id', 'type', 'locality', 'activation_date', 'latitude', 'longitude',
       'lease_type', 'gym', 'lift', 'swimming_pool', 'negotiable',
       'furnishing', 'parking', 'property_size', 'property_age', 'bathroom',
       'facing', 'cup_board', 'floor', 'total_floor', 'amenities',
       'water_supply', 'building_type', 'balconies', 'rent'],
      dtype='object')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20500 entries, 0 to 20499
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               20500 non-null  object 
 1   type             20500 non-null  object 
 2   locality         20500 non-null  object 
 3   activation_date  20500 non-null  object 
 4   latitude         20500 non-null  float64
 5   longitude        20500 non-null  float64
 6   lease_type       20500 non-null  object 
 7   gym              20500 non-null  int64  
 8   lift             20500 non-null  int64  
 9   swimming_pool    20500 non-null  int64  
 10  negotiable       20500 non-null  int64  
 11  furnishing       20500 non-null  object 
 12  parking          20500 non-null  object 
 13  property_size    20500 non-null  int64  
 14  property_age     20500 non-null  int64  
 15  bathroom         20500 non-null  int64  
 16  facing           20500 non-null  object 
 17  cup_board   

In [9]:
    le = LabelEncoder()
    label = le.fit_transform(data['id'])
    data.drop('id', axis=1, inplace=True)
    data["id"] = label
    data = data.drop(['id'],axis=1)
    data.head()

,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,...,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent
0,BHK2,"Cauvery Colony,Koramangala",21-04-2018 14:44,12.934471,77.634471,FAMILY,1,1,1,0,...,2,E,2,6,12,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",CORP_BORE,AP,2,40000
1,BHK2,Bellandur,09-10-2017 12:48,12.929557,77.672280,ANYONE,0,1,0,1,...,2,NE,2,3,4,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORPORATION,AP,2,22000
2,BHK3,Thiruvanmiyur,12-06-2018 22:02,12.982870,80.262012,FAMILY,0,1,0,0,...,3,E,3,1,5,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORP_BORE,AP,3,28000
3,BHK1,Attiguppe,20-07-2018 16:22,12.955991,77.531634,FAMILY,0,0,0,1,...,1,E,1,1,2,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,0,8000
4,BHK3,Kodihalli,31-05-2018 17:41,12.963903,77.649446,FAMILY,0,0,0,1,...,3,E,4,0,0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,1,45000


In [10]:
    label = le.fit_transform(data['activation_date'])
    data.drop('activation_date', axis=1, inplace=True)
    data["activation_date"] = label
    data = data.drop(['activation_date'],axis=1)

In [11]:
    label = le.fit_transform(data['amenities'])
    data.drop('amenities', axis=1, inplace=True)
    data["amenities"] = label
    data = data.drop(['amenities'],axis=1)

In [12]:
    label = le.fit_transform(data['locality'])
    data.drop('locality', axis=1, inplace=True)
    data["locality"] = label
    data = data.drop(['locality'],axis=1)

In [13]:
    label = le.fit_transform(data['longitude'])
    data.drop('longitude', axis=1, inplace=True)
    data["longitude"] = label
    data = data.drop(['longitude'],axis=1)

In [14]:
data['type']=data['type'].map({'BHK1':1,'BHK2':2,'BHK3':3})
data.head()

,type,latitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,12.934471,FAMILY,1,1,1,0,SEMI_FURNISHED,BOTH,1250,25,2,E,2,6,12,CORP_BORE,AP,2,40000
1,2.0,12.929557,ANYONE,0,1,0,1,SEMI_FURNISHED,BOTH,1400,4,2,NE,2,3,4,CORPORATION,AP,2,22000
2,3.0,12.982870,FAMILY,0,1,0,0,SEMI_FURNISHED,BOTH,1350,6,3,E,3,1,5,CORP_BORE,AP,3,28000
3,1.0,12.955991,FAMILY,0,0,0,1,SEMI_FURNISHED,TWO_WHEELER,600,3,1,E,1,1,2,CORPORATION,IH,0,8000
4,3.0,12.963903,FAMILY,0,0,0,1,SEMI_FURNISHED,BOTH,1500,15,3,E,4,0,0,CORPORATION,IH,1,45000


In [15]:
    label = le.fit_transform(data['latitude'])
    data.drop('latitude', axis=1, inplace=True)
    data["latitude"] = label
    data = data.drop(['latitude'],axis=1)

In [16]:
data['furnishing']=data['furnishing'].map({'NOT_FURNISHED':0,'SEMI_FURNISHED':1,'FULLY_FURNISHED':2})
data.head()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,FAMILY,1,1,1,0,1,BOTH,1250,25,2,E,2,6,12,CORP_BORE,AP,2,40000
1,2.0,ANYONE,0,1,0,1,1,BOTH,1400,4,2,NE,2,3,4,CORPORATION,AP,2,22000
2,3.0,FAMILY,0,1,0,0,1,BOTH,1350,6,3,E,3,1,5,CORP_BORE,AP,3,28000
3,1.0,FAMILY,0,0,0,1,1,TWO_WHEELER,600,3,1,E,1,1,2,CORPORATION,IH,0,8000
4,3.0,FAMILY,0,0,0,1,1,BOTH,1500,15,3,E,4,0,0,CORPORATION,IH,1,45000


In [17]:
data['parking']=data['parking'].map({'NONE':0,'TWO_WHEELER':2,'FOUR_WHEELER':4,'BOTH':1})
data.head()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,FAMILY,1,1,1,0,1,1,1250,25,2,E,2,6,12,CORP_BORE,AP,2,40000
1,2.0,ANYONE,0,1,0,1,1,1,1400,4,2,NE,2,3,4,CORPORATION,AP,2,22000
2,3.0,FAMILY,0,1,0,0,1,1,1350,6,3,E,3,1,5,CORP_BORE,AP,3,28000
3,1.0,FAMILY,0,0,0,1,1,2,600,3,1,E,1,1,2,CORPORATION,IH,0,8000
4,3.0,FAMILY,0,0,0,1,1,1,1500,15,3,E,4,0,0,CORPORATION,IH,1,45000


In [18]:
data['lease_type']=data['lease_type'].map({'FAMILY':0,'ANYONE':1,'BACHELOR':2,'COMPANY':3})
data.head()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,0,1,1,1,0,1,1,1250,25,2,E,2,6,12,CORP_BORE,AP,2,40000
1,2.0,1,0,1,0,1,1,1,1400,4,2,NE,2,3,4,CORPORATION,AP,2,22000
2,3.0,0,0,1,0,0,1,1,1350,6,3,E,3,1,5,CORP_BORE,AP,3,28000
3,1.0,0,0,0,0,1,1,2,600,3,1,E,1,1,2,CORPORATION,IH,0,8000
4,3.0,0,0,0,0,1,1,1,1500,15,3,E,4,0,0,CORPORATION,IH,1,45000


In [19]:
data['facing']=data['facing'].map({'N':0,'NE':1,'NW':2,'E':3,'SE':4,'S':5,'SW':6,'W':7})
data.head()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,0,1,1,1,0,1,1,1250,25,2,3,2,6,12,CORP_BORE,AP,2,40000
1,2.0,1,0,1,0,1,1,1,1400,4,2,1,2,3,4,CORPORATION,AP,2,22000
2,3.0,0,0,1,0,0,1,1,1350,6,3,3,3,1,5,CORP_BORE,AP,3,28000
3,1.0,0,0,0,0,1,1,2,600,3,1,3,1,1,2,CORPORATION,IH,0,8000
4,3.0,0,0,0,0,1,1,1,1500,15,3,3,4,0,0,CORPORATION,IH,1,45000


In [20]:
data['water_supply']=data['water_supply'].map({'CORP_BORE':0,'CORPORATION':1,'BOREWELL':2})
data.head()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,0,1,1,1,0,1,1,1250,25,2,3,2,6,12,0,AP,2,40000
1,2.0,1,0,1,0,1,1,1,1400,4,2,1,2,3,4,1,AP,2,22000
2,3.0,0,0,1,0,0,1,1,1350,6,3,3,3,1,5,0,AP,3,28000
3,1.0,0,0,0,0,1,1,2,600,3,1,3,1,1,2,1,IH,0,8000
4,3.0,0,0,0,0,1,1,1,1500,15,3,3,4,0,0,1,IH,1,45000


In [21]:
data['building_type']=data['building_type'].map({'AP':0,'IF':1,'IH':2,'GC':3})
data.head()

,type,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,property_size,property_age,bathroom,facing,cup_board,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,0,1,1,1,0,1,1,1250,25,2,3,2,6,12,0,0,2,40000
1,2.0,1,0,1,0,1,1,1,1400,4,2,1,2,3,4,1,0,2,22000
2,3.0,0,0,1,0,0,1,1,1350,6,3,3,3,1,5,0,0,3,28000
3,1.0,0,0,0,0,1,1,2,600,3,1,3,1,1,2,1,2,0,8000
4,3.0,0,0,0,0,1,1,1,1500,15,3,3,4,0,0,1,2,1,45000


In [22]:
    label = le.fit_transform(data['negotiable'])
    data.drop('negotiable', axis=1, inplace=True)
    data["negotiable"] = label
    data = data.drop(['negotiable'],axis=1)

In [23]:
    label = le.fit_transform(data['property_age'])
    data.drop('property_age', axis=1, inplace=True)
    data["property_age"] = label
    data = data.drop(['property_age'],axis=1)

In [24]:
    label = le.fit_transform(data['cup_board'])
    data.drop('cup_board', axis=1, inplace=True)
    data["cup_board"] = label
    data = data.drop(['cup_board'],axis=1)

In [103]:
data.rename(columns = {'type':'type_of_house'},inplace=True)

In [104]:
data.head()

,type_of_house,lease_type,gym,lift,swimming_pool,furnishing,parking,property_size,bathroom,facing,floor,total_floor,water_supply,building_type,balconies,rent
0,2.0,0,1,1,1,1,1,1250,2,3,6,12,0,0,2,40000
1,2.0,1,0,1,0,1,1,1400,2,1,3,4,1,0,2,22000
2,3.0,0,0,1,0,1,1,1350,3,3,1,5,0,0,3,28000
3,1.0,0,0,0,0,1,2,600,1,3,1,2,1,2,0,8000
4,3.0,0,0,0,0,1,1,1500,3,3,0,0,1,2,1,45000


## Training and Test Data split

In [105]:
target = np.array(data.drop(['rent'],1))
features = np.array(data['rent'])

In [106]:
target[0]

array([2.0, 0, 1, 1, 1, 1, 1, 1250, 2, 3, 6, 12, 0, 0, 2], dtype=object)

In [107]:
x_train , x_test , y_train , y_test = train_test_split(target,features,test_size=0.25,random_state=42)

len(x_train)/len(features)

0.75

In [108]:
print(f'Shape of x_train is {x_train.shape}')
print(f'Shape of x_test is {x_test.shape}')
print(f'Shape of y_train is {y_train.shape}')
print(f'Shape of y_test is {y_test.shape}')

Shape of x_train is (15375, 15)
Shape of x_test is (5125, 15)
Shape of y_train is (15375,)
Shape of y_test is (5125,)


In [109]:
data.isnull().sum()

type_of_house    0
lease_type       0
gym              0
lift             0
swimming_pool    0
furnishing       0
parking          0
property_size    0
bathroom         0
facing           0
floor            0
total_floor      0
water_supply     0
building_type    0
balconies        0
rent             0
dtype: int64

In [110]:
data.isna().sum().sum()

0

In [112]:
data['type_of_house'].fillna('1',inplace=True)

In [113]:
data.isna().sum().sum()

0

In [114]:
from sklearn.ensemble import HistGradientBoostingRegressor
gb = HistGradientBoostingRegressor()

In [115]:
gb.fit(x_train,y_train)

HistGradientBoostingRegressor()

In [116]:
#y_pred = [y_train.mean()] * len(y_train)
#print(mean_squared_error(y_train,y_pred))

71440106.9098202


In [119]:
y_pred = gb.predict(x_test)
gb_base = np.sqrt(mean_squared_error(y_test, y_pred))
gb_base

4718.815708644339

In [120]:
from sklearn import metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
r2 = metrics.r2_score(y_test, y_pred)

results = pd.DataFrame([['Multiple Linear Regression', mae, mse, rmse, r2]],
               columns = ['Model', 'MAE', 'MSE', 'RMSE', 'R2 Score'])

In [121]:
from sklearn.ensemble import HistGradientBoostingRegressor
gb_regressor = HistGradientBoostingRegressor()
gb_regressor.fit(x_train, y_train)

# Predicting Test Set
y_pred = gb_regressor.predict(x_test)
from sklearn import metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
r2 = metrics.r2_score(y_test, y_pred)

model_results = pd.DataFrame([['GradientBoosting Regressor', mae, mse, rmse, r2]],
               columns = ['Model', 'MAE', 'MSE', 'RMSE', 'R2 Score'])

results = results.append(model_results, ignore_index = True)

In [122]:
from xgboost import XGBRegressor
xgb_regressor = XGBRegressor()
xgb_regressor.fit(x_train, y_train)

# Predicting Test Set
y_pred = xgb_regressor.predict(x_test)
from sklearn import metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
r2 = metrics.r2_score(y_test, y_pred)

model_results = pd.DataFrame([['XGB Regressor', mae, mse, rmse, r2]],
               columns = ['Model', 'MAE', 'MSE', 'RMSE', 'R2 Score'])

results = results.append(model_results, ignore_index = True)

In [123]:
#The Best Regressor 
print('The best regressor is:')
print('{}'.format(results.sort_values(by='R2 Score',ascending=False).head(5)))

The best regressor is:
                        Model          MAE           MSE         RMSE  \
1  GradientBoosting Regressor  3462.424536  2.220634e+07  4712.359918   
0  Multiple Linear Regression  3452.198470  2.226722e+07  4718.815709   
2               XGB Regressor  3501.190246  2.312162e+07  4808.494123   

   R2 Score  
1  0.683552  
0  0.682684  
2  0.670509  


In [124]:
print('Training Accuracy:' ,gb_regressor.score(x_train,y_train))
print('Testing Accuracy:' ,gb_regressor.score(x_test,y_test))

Training Accuracy: 0.7355738379306975
Testing Accuracy: 0.6835517093023568


In [125]:
import pickle
pickle.dump(gb_regressor,open('housing.pkl','wb'))

In [126]:
model = pickle.load(open('housing.pkl','rb'))

In [127]:
print(gb_regressor.predict([[2,0,0,0,0,1,2,3000,2,0,5,5,0,0,1]]))

[26098.61585852]


In [129]:
y_pred = pd.DataFrame(y_pred,columns=['rent'])

In [133]:
output = pd.DataFrame({
"Rent": y_pred
})

ValueError: If using all scalar values, you must pass an index